In [1]:
import folium
import branca

import pandas
import numpy

from operator import itemgetter
from math import sin, cos, sqrt, atan2, radians

from VisualisationElementsProvider import Scenario
from VisualisationElementsProvider import VisualisationElementsProvider

import locale
locale.setlocale(locale.LC_ALL, 'en_US')

'en_US'

In [2]:
def calculateDistance(point1, point2):
    radius = 6373.0

    dlon = point2['lon'] - point1['lon']
    dlat = point2['lat'] - point1['lat']

    a = sin(dlat / 2)**2 + cos(point1['lat']) * cos(point2['lat']) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return radius * c

def findResultIndex(results, segment): 
    for i in range(len(results)): 
        if results[i]['road'] == segment['road'] and results[i]['LRPName'] == segment['LRPName'] : 
            return i
    
    return -42

In [3]:
def build_indicator_popup(segment): 
    vulnerability = -42
    traffic = -42
    membership = []
    road = ''
    lrp = ''
    for elem in segment:
        road = elem['road']
        lrp = elem['LRPName']
        traffic = elem['TotalTraffic']
        if vulnerability < elem['TotalEconomicVulnerability']: 
            vulnerability = elem['TotalEconomicVulnerability']
        membership.append(elem['scenario'])
        
    html = '<b>Road: </b>' + road + \
        '<br>' + \
        '<b>Key LRP: </b>' + lrp + \
        '<br>' + \
        '<b>Total Economic Traffic (per lane, per day): </b>' + locale.format("%d", traffic, grouping=True) + \
        '<br><br>' + \
        '<b>Maximum Economic Traffic Loss Risk (per lane, per day): </b>' + locale.format("%d", vulnerability, grouping=True) + \
        '<br><br>' + \
        '<b>Vulnerable under following damage scenarios: </b>' + \
        '<br>' + \
        '[' + ', '.join(membership) + ']'
    iframe = branca.element.IFrame(html=html, width=300, height=180)
    popup = folium.Popup(iframe, max_width=500)
    
    return popup

def build_road_popup(elem): 
    road = elem['road']
    lrp = elem['LRPName']
    traffic = elem['TotalTraffic']
    vulnerability = elem['TotalEconomicVulnerability']
            
    html = '<b>Road: </b>' + road + \
        '<br>' + \
        '<b>Key LRP: </b>' + lrp + \
        '<br>' + \
        '<b>Total Economic Traffic (per lane, per day): </b>' + locale.format("%d", traffic, grouping=True) + \
        '<br><br>' + \
        '<b>Maximum Economic Traffic Loss Risk (per lane, per day): </b>' + locale.format("%d", vulnerability, grouping=True)
    iframe = branca.element.IFrame(html=html, width=300, height=180)
    popup = folium.Popup(iframe, max_width=500)
    
    return popup

## Build the vulnerability ratings lists

Segment resolution refers to the lengh of segments being analyzed. High resolution will provide data about smaller road segments but will take longer to process, while low resolution calculates vulnerability over a longer road segment but will not take as long to calculate. 

The number of segments indicates how many of the most vulnerable segments that you wish to visualize. 

If you would like to run a more rapid version of the vulnerability algorithm, change the mode in `VisualElementsProvider` to `'short'`, which will return the most vulnerable segments of the N1, N2, N3, N4, and N5 only. 

If you would like to run a thorough calculation with all vulnerability elements tested, switch `mode` to `'all'`. 

Finally, to perform vulnerability calcuations on an entire class of roads (N, R, Z), mode can be set to a list of characters of all desired classes. For example, `'N'` for just N class roads, or `'NR'` to analyze both N and R class roads. 

You also have the option of adapting the bridge vulnerability scenarios to any value, based on the condition of the bridge (1:a, 2:b, 3:c, 4:d). 

In [6]:
segmentResolution = input('What road segment resolution would you like to analyze [low, medium, high]:')

What road segment resolution would you like to analyze [low, medium, high]:high


In [7]:
numberSegments = input('Please provide the number of vulnerable segments to visualize for each scenario: ')

Please provide the number of vulnerable segments to visualize for each scenario: 20


In [8]:
scenarios = [
    Scenario('linear', 0.2, 0.4, 0.6, 0.8),
    Scenario('log', 0.2, 0.49, 0.67, 0.8),
    Scenario('exp', 0.1, 0.2, 0.4, 0.8)
]

resolution = {
    'low': 100, 
    'medium': 75, 
    'high': 40
}

res = {}

selected_resolution = resolution[segmentResolution.lower()]
mode = 'NR'
numberSegments = int(numberSegments)

In [ ]:
res = VisualisationElementsProvider(scenarios, numberSegments, selected_resolution, mode=mode).provide()

In [ ]:
#Optional saving scenario results to 3 csvs
for key in res: 
    pandas.DataFrame.from_dict(res[key]).to_csv(key+'.csv')

In [9]:
#Optional reading scenario analysis from csvs
for scenario in scenarios: 
    df = pandas.DataFrame.from_csv(scenario.name + '.csv')
    res[scenario.name] = df.to_dict('records')

## Build Roads

In [10]:
roads = pandas.read_csv('../../WBSIM/infrastructure/_roads3.csv')

currResult = 0

roadLines = []
kw = dict(opacity=1.0, weight=2)

results = numpy.array(res['exp'])
results_sorted = sorted(results, key=lambda x: x['TotalEconomicVulnerability'], reverse=True)

prevRow = None
activeRoad = ''
currResultSet = []

onethird = int(len(results_sorted)/3)
twothirds = int(len(results_sorted)/3) * 2

road_segments = {'N':[],'R':[],'Z':[]}
for index, road in roads.iterrows():
    if activeRoad != road['road']: 
        activeRoad = road['road']
        
        newResults = []
        currResultSet = []
        for result in results: 
            if result['road'] == activeRoad: 
                currResultSet.append(result)
            else: 
                newResults.append(result)
        results = newResults
        
        currResult = 0
        
    if len(currResultSet) != 0:
        if len(currResultSet) > currResult+1: 
            currRes = currResultSet[currResult]
            nextRes = currResultSet[currResult+1]
            distanceFromResult = calculateDistance({'lat':road['lat'],'lon':road['lon']},
                                                   {'lat':currRes['Latitude'],'lon':currRes['Longitude']})
            distanceFromNextResult = calculateDistance({'lat':road['lat'],'lon':road['lon']},
                                                       {'lat':nextRes['Latitude'],'lon':nextRes['Longitude']})
                        
            if distanceFromNextResult < distanceFromResult: 
                currResult += 1
        activeResult = currResultSet[currResult]
    else: 
        activeResult = None
    
    if prevRow is not None and prevRow['road'] == activeRoad: 
        color = 'black'
        if activeResult is not None: 
            resultIndex = findResultIndex(results_sorted, activeResult)
            if resultIndex < onethird: 
                color = 'red'
            elif resultIndex < twothirds: 
                color = 'yellow'
            else: 
                color = 'green'
            road_segments[activeRoad[:1]].append(folium.PolyLine(
                locations=[(prevRow['lat'],prevRow['lon']), (road['lat'],road['lon'])], 
                popup=build_road_popup(activeResult),
                color=color, **kw))
        
    prevRow = road

## Add vulnerability indicators

Red circles indicates a segment is vulnerable in all three bridge vulnerability scenarios tested, yellow indicates a road segment is only included in the most vulnerable list for two scenarios, and green indicates that a road segment is only included in the most vulnerable list for one scenario. 

In [11]:
red_kw = dict(radius=10, fill_color='red', fill_opacity=1)
yellow_kw = dict(radius=8, fill_color='yellow', fill_opacity=1)
green_kw = dict(radius=6, fill_color='green', fill_opacity=1)

indicator_results = res
to_display = {}
for key in indicator_results: 
    indicator_results[key] = sorted(indicator_results[key], key=lambda x: x['TotalEconomicVulnerability'], reverse=True)
    if len(indicator_results[key]) > numberSegments:
        indicator_results[key] = indicator_results[key][:numberSegments]
            
    for elem in indicator_results[key]: 
        elem['scenario'] = key
        if elem['road']+'_'+elem['LRPName'] not in to_display:
            to_display[elem['road']+'_'+elem['LRPName']] = [elem]
        else: 
            to_display[elem['road']+'_'+elem['LRPName']].append(elem)
    
indicators = []
for key in to_display: 
    kwargs = yellow_kw
    if len(to_display[key]) == 1: 
        kwargs = green_kw
    elif len(to_display[key]) == len(scenarios): 
        kwargs = red_kw
        
    indicators.append(folium.CircleMarker(location=[to_display[key][0]['Latitude'], to_display[key][0]['Longitude']],
                                          popup=build_indicator_popup(to_display[key]),**kwargs))

## Build Visualization Map

In [12]:
min_lon, max_lon = 85, 96
min_lat, max_lat = 18, 30

vulnerability_map = folium.Map(location=[23.6925117, 90.3160594], 
                    tiles='Stamen Toner', 
                    zoom_start=6, 
                    min_lat=min_lat,
                    max_lat=max_lat,
                    min_lon=min_lon,
                    max_lon=max_lon,
                    width='100%',
                    height='100%')

#### Map with N road details

In [13]:
for road in road_segments['N']: 
    road.add_to(vulnerability_map)

#### Map with R road details

In [20]:
for road in road_segments['R']: 
    road.add_to(vulnerability_map)

#### Map with Z road details

In [ ]:
for road in road_segments['Z']: 
    road.add_to(vulnerability_map)

#### Most vulnerable segment indicators

In [14]:
for indicator in indicators: 
    indicator.add_to(vulnerability_map)

# Save the map

Once the map is saved, the HTML file can be opened in the browser for interaction. You are also able to view the map directly in this notebook, but the detail included leads to performance problems that can be avoided by opening the saved map in a separate window

In [15]:
vulnerability_map.save('vulnerabilityMap.html')